In [1]:
from qiskit import QuantumCircuit, transpile
import numpy as np

num_qubits = 8
qc = QuantumCircuit(num_qubits)
num_layers = int(np.ceil(np.log2(num_qubits)))


In [2]:
from qcc.ml import USE_CUDA

USE_CUDA


False

In [3]:
from itertools import zip_longest, tee

def convolve(wires):
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    
    qc.cnot(wires[0], wires[1])
    qc.ry(0.1, wires[0])
    qc.rz(0.1, wires[1])
    qc.cnot(wires[1], wires[0])
    qc.ry(0.1, wires[0])
    qc.cnot(wires[0], wires[1])
    
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
        
def pool(wires):
    qc.crz(0.1, wires[0], wires[1])
    qc.x(wires[0])
    qc.crx(0.1, wires[0], wires[1])

def _convolution(iterable):
    a, b = tee(iterable)
    first = next(b, None)
    lst = list(zip_longest(a, b, fillvalue=first))
    last = lst.pop()[::-1]
    lst = lst[::2] + lst[1::2]

    if len(lst) > 1:
        lst.insert(0, last)

    for wires in lst:
        convolve(wires)

def _pooling(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        pool(wires)

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    _convolution(wires)
    wires = _pooling(wires)


In [4]:
qc.draw('mpl')


KeyboardInterrupt: 

In [ ]:
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


(135, 415)

In [ ]:
135, 415
188, 593


(188, 593)

In [ ]:
qc = QuantumCircuit(num_qubits)

def _ry_pool(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        qc.cry(0.1, wires[0], wires[1])

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    wires = _ry_pool(wires)
    
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


(12, 28)

In [ ]:
import plotly.io as pio

pio.templates["plotly"]


layout.Template({
    'data': {'bar': [{'error_x': {'color': '#2a3f5f'},
                      'error_y': {'color': '#2a3f5f'},
                      'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
                                 'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
                      'type': 'bar'}],
             'barpolar': [{'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
                                      'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
                           'type': 'barpolar'}],
             'carpet': [{'aaxis': {'endlinecolor': '#2a3f5f',
                                   'gridcolor': 'white',
                                   'linecolor': 'white',
                                   'minorgridcolor': 'white',
                                   'startlinecolor': '#2a3f5f'},
                         'baxis': {'endlinecolor': '#2a3f5f',
                                   'gridcolor': 'white',


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

from qcc.file import load_toml

template = go.layout.Template(load_toml("uwu.toml"))

pio.templates["kuarq"] = template
pio.templates.default = "plotly+kuarq"


fig = go.Figure(
data=[
        go.Scatter(x=[0, 2, 3], y=[1, 3, 2], name="test"),
        go.Scatter(x=[1, 2, 3], y=[3, 2, 1], name="test2")
    ],
layout=go.Layout(
title=go.layout.Title(text="A Figure Specified By A Graph Object"),
xaxis=go.layout.XAxis(title_text="test"),
yaxis=go.layout.YAxis(title_text="test"),
    )
)

fig.show()


In [ ]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Downloads/results_fashionmnist")

for file in path.glob("**/*fashionmnist*.*"):
    items = file.stem.split("_")
    items.insert(2, "16x16")
    stem = "_".join(items)
    
    items = file.parent.stem.split("_")
    items.insert(2, "16x16")
    parent = "_".join(items)
    
    new_path = file.parent.with_stem(parent)
    new_path.mkdir(parents=True, exist_ok=True)
    
    new_path = new_path / f"{stem}{file.suffix}"
    file.rename(new_path)


In [15]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/test/2023-11-06")

for file in path.glob("**/*avgPool*copy.*"):
    stem = file.stem[:-5].replace("avgPool", "2NormPool")
    new_path = file.with_stem(stem)
    file.rename(new_path)
